## Text Summarization Evaluation - Abstractive and Extractive Summarization

This sample shows how use the evaluate a group of models against a given set of metrics for the `text-summarization` task. 

### Evaluation dataset
The CNN / DailyMail Dataset is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail. The current version supports both extractive and abstractive summarization, though the original version was created for machine reading and comprehension and abstractive question answering. A copy of the [cnn_dailymail](https://huggingface.co/datasets/cnn_dailymail) dataset is available in the [cnn_dailymail](./cnn_dailymail) folder.

### Model
The goal of evaluating models is to compare their performance on a variety of metrics. `text-summarization` is generic task type that can be used for scenarios such as abstractive and extractive summarization. As such, the models you pick to compare must be finetuned for same scenario. Given that we have the CNN_Dailymail dataset, we would like to look for models finetuned for this specific scenario. We will compare `sshleifer-distilbart-cnn-12-6`, and `facebook-bart-large-cnn` in this sample, which are available in the `azureml` system registry.

If you'd like to evaluate models that are not in the system registry, you can import those models to your workspace or organization registry and then evaluate them using the approach outlined in this sample. Review the sample notebook for [importing models](../../import/import-model-from-huggingface.ipynb). 

### Outline
* Setup pre-requisites such as compute.
* Pick the models to evaluate.
* Pick and explore evaluate data.
* Configure the evaluation jobs.
* Run the evaluation jobs.
* Review the evaluation metrics. 

### 1. Setup pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry
* Set an optional experiment name
* Check or create compute. A single GPU node can have multiple GPU cards. For example, in one node of `Standard_ND40rs_v2` there are 8 NVIDIA V100 GPUs while in `Standard_NC12s_v3`, there are 2 NVIDIA V100 GPUs. Refer to the [docs](https://learn.microsoft.com/en-us/azure/virtual-machines/sizes-gpu) for this information. The number of GPU cards per node is set in the param `gpus_per_node` below. Setting this value correctly will ensure utilization of all GPUs in the node. The recommended GPU compute SKUs can be found [here](https://learn.microsoft.com/en-us/azure/virtual-machines/ncv3-series) and [here](https://learn.microsoft.com/en-us/azure/virtual-machines/ndv2-series).

Install dependencies by running below cell. This is not an optional step if running in a new environment.

In [1]:
import pandas as pd
%pip install --upgrade azure-ai-ml
%pip install --upgrade azure-identity
%pip install --upgrade datasets==2.9.0

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = None
try:
    workspace_ml_client = MLClient.from_config(credential)
    subscription_id = workspace_ml_client.subscription_id
    workspace = workspace_ml_client.workspace_name
    resource_group = workspace_ml_client.resource_group_name
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"
    workspace_ml_client = MLClient(credential, subscription_id, resource_group, workspace)

# the models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml-preview"
preview_registry = "azureml-preview"

registry_ml_client = MLClient(credential, subscription_id, resource_group, registry_name=preview_registry)
registry_ml_client

Found the config file in: /config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7fb6cff276d0>,
         subscription_id=6c6683e9-e5fe-4038-8519-ce6ebec2ba15,
         resource_group_name=registry-builtin-preview-eastus,
         workspace_name=None)

In [3]:
# If you already have a gpu cluster, mention it here. Else will create a new one with the name 'gpu-cluster-big'
compute_cluster = "gpu-cluster" # -big"
try:
    compute = workspace_ml_client.compute.get(compute_cluster)
    print(f"GPU compute '{compute_cluster}' found.")
except Exception as ex:
    print(f"GPU compute '{compute_cluster}' not found. Creating new one.")
    compute = AmlCompute(
        name = compute_cluster,
        size= "Standard_ND40rs_v2",
        max_instances= 2 # For multi node training set this to an integer value more than 1
    )
    workspace_ml_client.compute.begin_create_or_update(compute).wait()

# genrating a unique timestamp that can be used for names and versions that need to be unique
timestamp = str(int(time.time()))

GPU compute 'gpu-cluster' found.


Below snippet will allow us to query number of GPU's present on the compute. We can use it to set `gpu_per_node` to ensure utilization of all GPUs in the node.

In [4]:
# This is the number of GPUs in a single node of the selected 'vm_size' compute.
# Setting this to less than the number of GPUs will result in underutilized GPUs, taking longer to train.
# Setting this to more than the number of GPUs will result in an error.
gpus_per_node = 1 # default value
gpu_count_found = False
ws_computes = workspace_ml_client.compute.list_sizes()
for ws_compute in ws_computes:
    if ws_compute.name.lower() == compute.size.lower():
        gpu_count_found = True
        gpus_per_node = ws_compute.gpus
        print(f"Number of GPU's in compute {ws_compute.name} are {ws_compute.gpus}")
# if gpu_count_found not found, then print an error
gpu_count_found or print(f"Error: Number of GPU's in compute {compute.size} not found")

Number of GPU's in compute Standard_NV6 are 1


True

### 2. Pick the models to evaluate

Verify that the models selected for evaluation are available in system registry

In [23]:
# need to specify model versions until the bug to support fetching the latest version using latest label is fixed
models = [
    {'name': 'sshleifer-distilbart-cnn-12-6', 'version': '3'},
    {'name': 'facebook-bart-large-cnn', 'version': '3'},
]
for model in models:
    model = registry_ml_client.models.get(model['name'], version=model['version'])
    print (model.id)

azureml://registries/azureml-preview/models/sshleifer-distilbart-cnn-12-6/versions/3
azureml://registries/azureml-preview/models/facebook-bart-large-cnn/versions/3


### 3. Pick the test dataset for evaluation
A copy of the cnn_dailymail is available in the [cnn_dailymail](./cnn_dailymail/) folder. The next few cells show basic data preparation:
* Visualize some data rows
* We want this sample to run quickly, so we use a smaller dataset containing 10% of the original.
* To use the entire dataset, uncomment the cells below and run.

In [41]:
# !pip install datasets

In [ ]:
# from datasets import load_dataset

# hf_test_data = load_dataset('cnn_dailymail')

# hf_test_data['train'].to_pandas().head()

In [24]:
test_data = './cnn_dailymail/small-test.jsonl'

In [ ]:
pd.read_json(test_data, lines=True).head()

### 4. Submit the evaluation jobs using the the model and data as inputs

Create the job that uses the `model_evaluation_pipeline` component. We will submit one job per model.

Note that the metrics that the evaluation jobs need to calculate are specified in the [eval_config.json](./eval_config.json) file. We calculate `accuracy`, `f1_score_micro` and `average_precision_score_macro` in this sample.

In [25]:
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

# fetch the pipeline component
pipeline_component_func = registry_ml_client.components.get(name="model_evaluation_pipeline", label="latest")

# define the pipeline job
@pipeline()
def evaluation_pipeline(
    mlflow_model
):
    evaluation_job = pipeline_component_func(
        # specify the foundation model available in the azureml system registry or a model from the workspace
       # mlflow_model = Input(type=AssetTypes.MLFLOW_MODEL, path=f"{mlflow_model_path}"),
        mlflow_model = mlflow_model,

        # test data
        test_data = Input(type=AssetTypes.URI_FILE, path=test_data),
        # The following parameters map to the dataset fields
        input_column_names = "input_string",
        label_column_name = "summary",
        # Evaluation settings
        task = "text-summarization",
        # config file containing the details of evaluation metrics to calculate
        evaluation_config = Input(type=AssetTypes.URI_FILE, path="./eval-config.json"),
        # config cluster/device job is running on
        device = "gpu",
    )
    return {
        "evaluation_result": evaluation_job.outputs.evaluation_result
    }

Submit the jobs, passing the model as a parameter to the pipeline created in the above step.

In [26]:
# submit the pipeline job for each model that we want to evaluate
# you could consider submitting the pipeline jobs in parallel, provided your cluster has multiple nodes
pipeline_jobs = []

experiment_name = "summarization-evaluation"

for model in models:
    model_object = registry_ml_client.models.get(model['name'], version=model['version'])
    pipeline_object = evaluation_pipeline(mlflow_model=Input(type=AssetTypes.MLFLOW_MODEL, path=f"{model_object.id}"),)
    # don't reuse cached results from previous jobs
    pipeline_object.settings.force_rerun = True
    pipeline_object.settings.default_compute = compute_cluster
    pipeline_object.display_name = f"eval-{model['name']}-{timestamp}"
    pipeline_job = workspace_ml_client.jobs.create_or_update(pipeline_object, experiment_name=experiment_name)
    # add model['name'] and pipeline_job.name as key value pairs to a dictionary
    pipeline_jobs.append({'model_name': model['name'], 'job_name': pipeline_job.name})
    # wait for the pipeline job to complete
    workspace_ml_client.jobs.stream(pipeline_job.name)

RunId: quirky_puppy_vmrpgg4nt3
Web View: https://ml.azure.com/runs/quirky_puppy_vmrpgg4nt3?wsid=/subscriptions/72c03bf3-4e69-41af-9532-dfcdc3eefef4/resourcegroups/shared-model-evaluation-rg/workspaces/aml-shared-model-evaluation-ws

Execution Summary
RunId: quirky_puppy_vmrpgg4nt3
Web View: https://ml.azure.com/runs/quirky_puppy_vmrpgg4nt3?wsid=/subscriptions/72c03bf3-4e69-41af-9532-dfcdc3eefef4/resourcegroups/shared-model-evaluation-rg/workspaces/aml-shared-model-evaluation-ws

RunId: tidy_engine_z51ghhstjy
Web View: https://ml.azure.com/runs/tidy_engine_z51ghhstjy?wsid=/subscriptions/72c03bf3-4e69-41af-9532-dfcdc3eefef4/resourcegroups/shared-model-evaluation-rg/workspaces/aml-shared-model-evaluation-ws

Streaming logs/azureml/executionlogs.txt

[2023-04-02 14:34:35Z] Submitting 1 runs, first five are: ac29aba6:83a64726-c290-4c52-ac9a-a408e7cf26bd
[2023-04-02 14:39:06Z] Completing processing run id 83a64726-c290-4c52-ac9a-a408e7cf26bd.

Execution Summary
RunId: tidy_engine_z51ghhstjy


### 5. Review evaluation metrics
Viewing the job in AzureML studio is the best way to analyze logs, metrics and outputs of jobs. You can create custom charts and compare metics across different jobs. See https://learn.microsoft.com/en-us/azure/machine-learning/how-to-log-view-metrics?tabs=interactive#view-jobsruns-information-in-the-studio to learn more.

However, we may need to access and review metrics programmatically for which we will use MLflow, which is the recommended client for logging and querying metrics.

In [27]:
import mlflow, json
mlflow_tracking_uri = workspace_ml_client.workspaces.get(workspace_ml_client.workspace_name).mlflow_tracking_uri
mlflow.set_tracking_uri(mlflow_tracking_uri)

metrics_df = pd.DataFrame()
for job in pipeline_jobs:
    # concat 'tags.mlflow.rootRunId=' and pipeline_job.name in single quotes as filter variable
    filter = "tags.mlflow.rootRunId='" + job['job_name'] + "'"
    runs = mlflow.search_runs(experiment_names=[ experiment_name ], filter_string = filter, output_format="list")
    # get the compute_metrics runs.
    # using a hacky way till 'Bug 2320997: not able to show eval metrics in FT notebooks - mlflow client now showing display names' is fixed
    for run in runs:
        # else, check if run.data.metrics.accuracy exists
        if 'rouge1' in run.data.metrics:
            # get the metrics from the mlflow run
            run_metric = run.data.metrics
            # add the model name to the run_metric dictionary
            run_metric['model_name'] = job['model_name']
            # convert the run_metric dictionary to a pandas dataframe
            temp_df = pd.DataFrame(run_metric,index=[0])
            # concat the temp_df to the metrics_df
            metrics_df = pd.concat([metrics_df, temp_df], ignore_index=True)

# move the model_name columns to the first column
cols = metrics_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
metrics_df = metrics_df[cols]
metrics_df.head()

,model_name,rouge1,rouge2,rougeL,rougeLsum
0,sshleifer-distilbart-cnn-12-6,0.195049,0.038071,0.130519,0.130886
1,facebook-bart-large-cnn,0.200203,0.037690,0.134666,0.134818
